<a href="https://colab.research.google.com/github/mferartola/OpenAlexInstitutions/blob/main/001_get_training_data_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##probando
import pickle
!pip install boto3
import boto3
!pip install fsspec
import fsspec
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
#sc = spark.sparkContext         # Since Spark 2.0 'spark' is a
#SparkSession object that is by default created upfront and available
#in Spark shell, you need to explicitly create SparkSession object by
#using builder
spark = SparkSession.builder.getOrCreate()
#sc = SparkContext().getOrCreate()
sc = SparkContext._active_spark_context #devuelve la instancia existente
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc,spark)

from google.colab import drive

import os

!pip install pyalex
import pyalex
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
from pyalex import config

!pip install colab-convert## comentar en script



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 3.9 MB/s eta 0:00:00
ERROR: Invalid requirement: 'colab-convert##': Expected end or semicolon (after name and no valid version specifier)
    colab-convert##
                 ^


In [ ]:
drive.mount('/content/drive') ### comentar en script

Mounted at /content/drive


**Cambiar path para correr desde otro lado**

In [ ]:
base_path = '/content/drive/MyDrive/openalex-institution-parsing/'

base_save_path = "/content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/"

iteration_save_path = "/content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/"


### Getting all data (From saved OpenAlex DB snapshot)

In [ ]:
#institutions = spark.read.parquet(f"{base_save_path}/OA_static_institutions_single_file.parquet") \
 #   .filter(F.col('ror_id')!='')

### ALTERNATIVA DE STATIC INSTITUTIONS: BAJAR CON PYALEX PARTE DEL OBJETO INSTITUTIONS

config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

pager = Institutions().filter(country_code="AR").paginate(method="page",per_page=200)

listaDeInstituciones = list()
for page in pager:
    print(len(page))
    listaDeInstituciones += page

print(len(listaDeInstituciones))

institutions_df=pd.DataFrame(listaDeInstituciones)
institutions_df['affiliation_id'] = institutions_df['id'].apply(lambda x: x.split("/")[-1])
institutions_df['affiliation_id']=institutions_df['affiliation_id'].apply(lambda x: x.split("I")[-1])
institutions_df['ror_id'] = institutions_df['ror'].apply(lambda x: x.split("/")[-1])
print(institutions_df.head(n=2))
print(institutions_df.shape)

200
131
0
331
                                id                        ror  \
0  https://openalex.org/I151201029  https://ror.org/03cqe8w59   
1   https://openalex.org/I24354313  https://ror.org/0081fs513   

                                        display_name country_code        type  \
0  Consejo Nacional de Investigaciones Científica...           AR  government   
1                         University of Buenos Aires           AR   education   

                                             type_id  \
0  https://openalex.org/institution-types/government   
1   https://openalex.org/institution-types/education   

                                             lineage  \
0  [https://openalex.org/I151201029, https://open...   
1                   [https://openalex.org/I24354313]   

                homepage_url  \
0  http://www.conicet.gov.ar   
1         http://www.uba.ar/   

                                           image_url  \
0  https://commons.wikimedia.org/w/index.php?titl...   

In [ ]:
#### STATIC AFFILIATION: SE BAJA DESDE OPENALEX CON PYALEX

pager = Works().filter(authorships={"institutions": {"country_code": "AR"}}).paginate(method="page",per_page=200)
listaDeWorks = list()
for page in pager:
    print(len(page))
    listaDeWorks += page

argentina_works=pd.DataFrame(listaDeWorks)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [ ]:
argentina_works.columns

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'ids', 'language', 'primary_location', 'type',
       'type_crossref', 'indexed_in', 'open_access', 'authorships',
       'institution_assertions', 'countries_distinct_count',
       'institutions_distinct_count', 'corresponding_author_ids',
       'corresponding_institution_ids', 'apc_list', 'apc_paid', 'fwci',
       'has_fulltext', 'cited_by_count', 'citation_normalized_percentile',
       'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext',
       'primary_topic', 'topics', 'keywords', 'concepts', 'mesh',
       'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'datasets', 'versions',
       'referenced_works_count', 'referenced_works', 'related_works',
       'abstract_inverted_index', 'cited_by_api_url', 'counts_by_year',
       'updated_date', 'created_date', 'fulltext_origin',
       'is_authors_truncated'],
      dtype=

In [ ]:
## de columna affiliations
argentina_works['affiliations'] = argentina_works['authorships'].apply(lambda x: [i['affiliations'] for i in x] if isinstance(x,list) else [])
argentina_works['raw_affiliation_string'] = argentina_works['affiliations'].apply(lambda x: [d.get('raw_affiliation_string') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works['institution_ids'] = argentina_works['affiliations'].apply(lambda x: [d.get('institution_ids') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
####
## de columna raw_affiliation_strings
argentina_works['raw_affiliation_strings'] = argentina_works['authorships'].apply(lambda x: [i['raw_affiliation_strings'] for i in x] if isinstance(x,list) else [])
####
### de columna institutions
argentina_works['institutions'] = argentina_works['authorships'].apply(lambda x: [i['institutions'] for i in x] if isinstance(x,list) else [])
argentina_works['country_code'] = argentina_works['institutions'].apply(lambda x: [d.get('country_code') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works['institutions_bis'] = argentina_works['institutions'].apply(lambda x: [d.get('id') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works['display_name'] = argentina_works['institutions'].apply(lambda x: [d.get('display_name') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])

In [ ]:
argentina_works=argentina_works[['id','institution_ids','institutions_bis','raw_affiliation_string','raw_affiliation_strings','display_name','country_code']]
print(argentina_works.head(2))


                                 id  \
0  https://openalex.org/W2746485780   
1  https://openalex.org/W2136489990   

                                     institution_ids  \
0  [[https://openalex.org/I149521062], [https://o...   
1  [[https://openalex.org/I2802834092, https://op...   

                                    institutions_bis  \
0  [https://openalex.org/I149521062, https://open...   
1  [https://openalex.org/I2802834092, https://ope...   

                              raw_affiliation_string  \
0  [Center for Environmental and Estuarine Studie...   
1  [Population Health Research Institute, McMaste...   

                             raw_affiliation_strings  \
0  [[Center for Environmental and Estuarine Studi...   
1  [[Population Health Research Institute, McMast...   

                                        display_name  \
0  [University of Maryland Center for Environment...   
1  [Population Health Research Institute, Hamilto...   

                                     

In [ ]:
affiliations=argentina_works
#affiliations=affiliations.set_index('id')


In [ ]:
affiliations.to_csv(f"{base_path}/Crudos/ArchivosPyAlex/affiliations.csv")

In [ ]:
affiliations.explode['institution_ids']=affiliations['institution_ids']
affiliations.explode['raw_affiliation_string']=affiliations['raw_affiliation_string']

affiliations=affiliations.explode('institution_ids','raw_affiliation_string').reset_index(drop=False)


TypeError: 'method' object does not support item assignment

In [ ]:
affiliations.sort_values(by='institution_ids').head(2)

,id,institution_ids,institutions_bis,raw_affiliation_string,raw_affiliation_strings,display_name,country_code
364,https://openalex.org/W2758688195,"[[], [], [], [], [], [], [], [], [], [https://...","[https://openalex.org/I4210085984, https://ope...","[HCor Research Institute, São Paulo, Brazil, H...","[[HCor Research Institute, São Paulo, Brazil],...","[Hospital Moinhos de Vento, Irmandade da Santa...","[BR, BR, BR, BR, BR, BR, BR, BR, CO, BR, CO, B..."
7028,https://openalex.org/W2898410112,"[[], [], [], [], [], [], [], [], [https://open...","[https://openalex.org/I899635006, https://open...","[MACVIA-France, Fondation Partenariale FMC VIA...","[[MACVIA-France, Fondation Partenariale FMC VI...","[Université Grenoble Alpes, Inserm, Universida...","[FR, FR, PT, PT, PT, PT, PT, PT, BR, BR, BR, I..."


In [ ]:
affiliations['works_id'] = affiliations['id'].apply(lambda x: x.split("/")[-1])
affiliations['affiliation_id'] = affiliations['institution_ids'].apply(lambda x: x.split("/")[-1] if isinstance(x, str) else None) # Check if x is a string
affiliations['affiliation_id']= affiliations['affiliation_id'].apply(lambda x: x.split("I")[-1] if isinstance(x, str) else None) # Check if x is a string
affiliations['original_affiliation']=affiliations['raw_affiliation_string']
affiliations.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       10000 non-null  object
 1   institution_ids          10000 non-null  object
 2   institutions_bis         10000 non-null  object
 3   raw_affiliation_string   10000 non-null  object
 4   raw_affiliation_strings  10000 non-null  object
 5   display_name             10000 non-null  object
 6   country_code             10000 non-null  object
 7   works_id                 10000 non-null  object
 8   affiliation_id           0 non-null      object
 9   original_affiliation     10000 non-null  object
dtypes: object(10)
memory usage: 781.4+ KB


In [ ]:
affiliations.head(3)

,id,institution_ids,institutions_bis,raw_affiliation_string,raw_affiliation_strings,display_name,country_code,works_id,affiliation_id,original_affiliation
0,https://openalex.org/W2746485780,"[[https://openalex.org/I149521062], [https://o...","[https://openalex.org/I149521062, https://open...",[Center for Environmental and Estuarine Studie...,[[Center for Environmental and Estuarine Studi...,[University of Maryland Center for Environment...,"[US, US, NL, US, US, US, US, US, US, AR, US, US]",W2746485780,None,[Center for Environmental and Estuarine Studie...
1,https://openalex.org/W2136489990,"[[https://openalex.org/I2802834092, https://op...","[https://openalex.org/I2802834092, https://ope...","[Population Health Research Institute, McMaste...","[[Population Health Research Institute, McMast...","[Population Health Research Institute, Hamilto...","[CA, CA, CA, US, CA, CA, CA, CA, CA, CA, US, C...",W2136489990,None,"[Population Health Research Institute, McMaste..."
2,https://openalex.org/W2144173184,"[[https://openalex.org/I4210100468], [https://...","[https://openalex.org/I4210100468, https://ope...","[Department of Surgery and Swiss HPB Center, U...","[[Department of Surgery and Swiss HPB Center, ...","[University Hospital of Zurich, McGill Univers...","[CH, CA, JP, CH, US, CH, US, AR, AR, CH, AU, C...",W2144173184,None,"[Department of Surgery and Swiss HPB Center, U..."


In [ ]:
affiliations.shape

(186641, 11)

In [ ]:
##Genero el archivo para testeo
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(affiliations, train_size=0.90, random_state=1)
affiliations = train_data.reset_index(drop=True).copy()
affiliations_para_testeo = val_data.reset_index(drop=True).copy()

In [ ]:
##Genero el archivo para testeo
affiliations.to_parquet(f"{base_path}/Crudos/ArchivosPyAlex/affiliations.parquet")
affiliations_para_testeo.to_csv(f"{base_path}/Crudos/ArchivosPyAlex/affiliations_para_testeo.csv")

In [ ]:
affiliations = spark.read.parquet(f"{base_path}/Crudos/ArchivosPyAlex/affiliations.parquet")

#### Getting ROR aff strings

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# If SparkSession is not already initialized, create it
if 'spark' not in locals():  # Check if 'spark' variable exists
    spark = SparkSession.builder.appName("YourAppName").getOrCreate()

In [ ]:
from pyspark.sql.functions import explode_outer, flatten

# Explode the 'original_affiliation' column, no need for flatten
dedup_affs = affiliations.withColumn("original_affiliation", explode_outer(F.col("original_affiliation"))) \
    .select(F.trim(F.col('original_affiliation')).alias('original_affiliation'), 'affiliation_id')\
    .filter(F.col('original_affiliation').isNotNull())\
    .filter(F.col('original_affiliation')!='')\
    .withColumn('aff_len', F.length(F.col('original_affiliation')))\
    .filter(F.col('aff_len')>2)\
    .groupby(['original_affiliation','affiliation_id']) \
    .agg(F.count(F.col('affiliation_id')).alias('aff_string_counts'))

In [ ]:
dedup_affs.cache().count()

78710

In [ ]:
dedup_affs

DataFrame[original_affiliation: string, affiliation_id: int, aff_string_counts: bigint]

In [ ]:
dedup_affs.head(n=3)

[Row(original_affiliation='Institute for Clinical Effectiveness and Health Policy (IECS-CONICET); Argentine Cochrane Centre; Dr. Emilio Ravignani 2024 Buenos Aires Capital Federal Argentina C1414CPV', affiliation_id=None, aff_string_counts=0),
 Row(original_affiliation='Laboratory of Experimental Immunology, University of Copenhagen, Copenhagen N, Denmark', affiliation_id=None, aff_string_counts=0),
 Row(original_affiliation='Study Center for Primary Immunodeficiencies, Necker Hospital for Sick Children, AP-HP, 75015 Paris, France', affiliation_id=None, aff_string_counts=0)]

In [ ]:
ror_data = spark.read.parquet(f"{base_path}V2/001_Exploration/ror_strings.parquet") \
.select('original_affiliation','affiliation_id')

In [ ]:
ror_data.cache().count()

45963

In [ ]:
ror_data.head(n=2)

[Row(original_affiliation='National University of the Littoral, Santa Fe, Argentina', affiliation_id='3121987637'),
 Row(original_affiliation='National University of the Littoral, Argentina', affiliation_id='3121987637')]

### Gathering training data

Since we are looking at all institutions, we need to up-sample the institutions that don't have many affiliation strings and down-sample the institutions that have large numbers of strings. There was a balance here that needed to be acheived. The more samples that are taken for each institution, the more overall training data we will have and the longer our model will take to train.

However, more samples also means more ways of an institution showing up in an affiliation string.

The number of samples was set to 50 as it was determined this was a good optimization point based on affiliation string count distribution and time it would take to train the model.

However, unlike in V1 where we tried to keep all institutions at 50, for V2 we gave additional samples for institutions with more strings available. Specifically, we allowed those institutions to have up to 25 additional strings, for a total of 75.

In [ ]:
num_samples_to_get = 50

In [ ]:
##se queda sólo con los papers cuyas instituciones aparecen en ROR
w1 = Window.partitionBy('affiliation_id')

filled_affiliations = dedup_affs \
    .join(ror_data.select('affiliation_id'), how='inner', on='affiliation_id') \
    .select('original_affiliation','affiliation_id')

print(filled_affiliations.head())
##opción: cambiar de 'inner' a 'left'

None


In [ ]:
## se agregan las otras instituticiones del ror en Argentina
filled_affiliations=filled_affiliations.union(ror_data.select('original_affiliation','affiliation_id'))
print(filled_affiliations.head())

Row(original_affiliation='National University of the Littoral, Santa Fe, Argentina', affiliation_id='3121987637')


In [ ]:
filled_affiliations=filled_affiliations.filter(~F.col('affiliation_id').isNull()) \
    .dropDuplicates() \
    .withColumn('random_prob', F.rand(seed=20)) \
    .withColumn('id_count', F.count(F.col('affiliation_id')).over(w1)) \
    .withColumn('scaled_count', F.lit(1)-((F.col('id_count') - F.lit(num_samples_to_get))/(F.lit(3500000) - F.lit(num_samples_to_get)))) \
    .withColumn('final_prob', F.col('random_prob')*F.col('scaled_count'))

In [ ]:
filled_affiliations.select('affiliation_id').distinct().count()

331

In [ ]:
filled_affiliations.head(n=2)

[Row(original_affiliation='Dept. of Thoracic and Cardiovascular Surgery University of Mendoza Argentina', affiliation_id='100084298', random_prob=0.39526509100711327, id_count=147, scaled_count=0.9999722853183617, final_prob=0.39525413636095325),
 Row(original_affiliation='Dept. of Sociology and Anthropology Universidad de Mendoza Argentina', affiliation_id='100084298', random_prob=0.7961501486951794, id_count=147, scaled_count=0.9999722853183617, final_prob=0.7961280836472721)]

In [ ]:
less_than = filled_affiliations.dropDuplicates(subset=['affiliation_id']).filter(F.col('id_count') < num_samples_to_get).toPandas()

less_than['affiliation_id'] = less_than['affiliation_id'].astype('str')
less_than['affiliation_id'] = less_than['affiliation_id'].apply(lambda x: x.split("/")[-1])

print(less_than.shape)
print(less_than.head(n=2))


(159, 6)
                         original_affiliation affiliation_id  random_prob  \
0                                         IGN     1333839041     0.530564   
1  Academia Nacional de Medicina Buenos Aires       14429277     0.447859   

   id_count  scaled_count  final_prob  
0        26      1.000007    0.530568  
1        26      1.000007    0.447862  


In [ ]:
temp_df_list = []
for aff_id in less_than['affiliation_id'].unique():
    temp_df = less_than[less_than['affiliation_id']==aff_id].copy()
    help_df = temp_df.sample(num_samples_to_get - temp_df.shape[0], replace=True)
    temp_df_list.append(pd.concat([temp_df, help_df], axis=0))
less_than_df = pd.concat(temp_df_list, axis=0)

In [ ]:
# only install fsspec and s3fs
less_than_df[['original_affiliation', 'affiliation_id']].to_parquet(f"{iteration_save_path}lower_than_{num_samples_to_get}.parquet")
w1 = Window.partitionBy('affiliation_id').orderBy('random_prob')

more_than = filled_affiliations.filter(F.col('id_count') >= num_samples_to_get) \
.withColumn('row_number', F.row_number().over(w1)) \
.filter(F.col('row_number') <= num_samples_to_get+25)

more_than.cache().count()
more_than=more_than.toPandas()

more_than['affiliation_id'] = more_than['affiliation_id'].astype('str')
more_than['affiliation_id'] = more_than['affiliation_id'].apply(lambda x: x.split("/")[-1])
more_than[['original_affiliation', 'affiliation_id']].to_parquet(f"{iteration_save_path}more_than_{num_samples_to_get}.parquet")

#more_than.select('original_affiliation', 'affiliation_id') \
#.coalesce(1).write.mode('overwrite').parquet(f"{iteration_save_path}more_than_{num_samples_to_get}")

In [ ]:
print(less_than.sample(5))

                                  original_affiliation affiliation_id  \
44    Fundacion Via Libre, Cordoba, Cordoba, Argentina     4210107725   
17                                    IFISUR Argentina     4210094172   
127  Ministerio Publico Fiscal de la Ciudad Autonom...     4210150669   
21                         IADT Buenos Aires Argentina     4210095433   
100             ITeDA, Buenos Aires, Buenos Aires F.D.     4210136977   

     random_prob  id_count  scaled_count  final_prob  
44      0.036966        11      1.000011    0.036966  
17      0.998913        26      1.000007    0.998919  
127     0.353591        13      1.000011    0.353595  
21      0.229268        26      1.000007    0.229270  
100     0.411280        26      1.000007    0.411283  


In [ ]:
less_than_df.shape

(7950, 6)

In [ ]:
print(more_than.head(5))

                                original_affiliation affiliation_id  \
0  Argentine Atlantis University Dept. of Mathema...     4210131417   
1  School of Chemical Engineering & Technology Ar...     4210131417   
2  Argentine Atlantis University Dep. of Surgery,...     4210131417   
3                      Argentine Atlantis University     4210131417   
4  Argentine Atlantis University Dep. of Politica...     4210131417   

   random_prob  id_count  scaled_count  final_prob  row_number  
0     0.004403       141      0.999974    0.004402           1  
1     0.004745       141      0.999974    0.004745           2  
2     0.010532       141      0.999974    0.010532           3  
3     0.015843       141      0.999974    0.015843           4  
4     0.020232       141      0.999974    0.020232           5  


In [ ]:
more_than.shape

(12653, 7)

In [ ]:
less_than.sort_values(by=['affiliation_id']).head(15)

,original_affiliation,affiliation_id,random_prob,id_count,scaled_count,final_prob
0,IGN,1333839041,0.530564,26,1.000007,0.530568
1,Academia Nacional de Medicina Buenos Aires,14429277,0.447859,26,1.000007,0.447862
2,CONICET Buenos Aires,151201029,0.003941,21,1.000008,0.003941
3,Fundacion Instituto Leloir Buenos Aires,2801749607,0.028276,13,1.000011,0.028276
4,Centro Atomico Bariloche Argentina,2883434090,0.037979,26,1.000007,0.037979
5,"Instituto Nacional de Tecnologia Industrial, B...",41147313,0.759744,39,1.000003,0.759746
6,MACN Argentina,4210086432,0.730008,39,1.000003,0.730010
7,"CIMEC, Santa Fe",4210087179,0.064041,21,1.000008,0.064042
8,"Centro de Estudios para el Lujo Sustentable, A...",4210087318,0.945327,26,1.000007,0.945334
9,Centro Cientifico Tecnologico del Consejo Naci...,4210087694,0.148214,21,1.000008,0.148215


In [ ]:
more_than.sort_values(by=['affiliation_id']).head()

,original_affiliation,affiliation_id,random_prob,id_count,scaled_count,final_prob,row_number
6451,"Department of Surgery Universidad de Mendoza, ...",100084298,0.551235,147,0.999972,0.551220,70
6403,Dep of Psychology UM Mendoza Argentina,100084298,0.191611,147,0.999972,0.191606,22
6404,Sch. of Design University of Mendoza Mendoza,100084298,0.196534,147,0.999972,0.196529,23
6405,University of Mendoza Mendoza Argentina,100084298,0.198086,147,0.999972,0.198081,24
6406,Dept of Psychology University of Mendoza Argen...,100084298,0.206674,147,0.999972,0.206669,25


In [ ]:
##comentar en script
##!colab-convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/001_get_training_data_SPARK.ipynb /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/001_get_training_data_SPARK.py

In [ ]:
print("Terminó")

Terminó
